# ⚠️ Found an error in tfidf and count word vectorizer in many baseline NB, this notebook is showing it and (try to) explain why ⚠️

In many baseline and high LB score public NB, there is probably an error in **word n-gram tokenizer**, that behaves like a **character n-gram tokenizer**.

To be noted that correcting the error **DOES NOT** improve the score, and you would probably want to keep char n-gram tokenizer or both ! 

* copied NB to show this error:
    * https://www.kaggle.com/code/yongsukprasertsuk/0-818-deberta-v3-large-lgbm-baseline 


* other kernels probably affected:
    * https://www.kaggle.com/code/hideyukizushi/aes2-deberta-lgbm-countvectorizer-lb-819
    * https://www.kaggle.com/code/olyatsimboy/81-1-aes2-5folddeberta-lgbm-stacking 
    * https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained

Many thanks 🙏 to those author for those great kernels !

In [ ]:
import pandas as pd 

from datasets import Dataset

import gc

from scipy.special import softmax

MAX_LENGTH = 1024
TEST_DATA_PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
MODEL_PATH = '/kaggle/input/aes2-400-20240419134941/*/*'
EVAL_BATCH_SIZE = 1

In [ ]:
# Importing necessary libraries
import pandas as pd 
from datasets import Dataset  

import gc
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import spacy
import string
import random
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier,BaggingClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.metrics import cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
from sklearn.linear_model import SGDClassifier
import polars as pl
import joblib

In [ ]:
columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]
PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"

# Load training and testing sets, while using \ n \ n character segmentation to list and renaming to paragraph for full_text data
train = pl.read_csv(PATH + "train.csv").with_columns(columns)
test = pl.read_csv(PATH + "test.csv").with_columns(columns)

nlp = spacy.load("en_core_web_sm")
with open('/kaggle/input/english-word-hx/words.txt', 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)

# Display the first sample data in the training set
train.head(1)

## Features engineering

In [ ]:
def count_spelling_errors(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_.lower() for token in doc]
    spelling_errors = sum(1 for token in lemmatized_tokens if token not in english_vocab)
    return spelling_errors

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

## Paragraph Features

In [ ]:
# paragraph features
def remove_punctuation(text):
    """
    Remove all punctuation from the input text.
    
    Args:
    - text (str): The input text.
    
    Returns:
    - str: The text with punctuation removed.
    """
    # string.punctuation
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def Paragraph_Preprocess(tmp):
    # Expand the paragraph list into several lines of data
    tmp = tmp.explode('paragraph')
    # Paragraph preprocessing
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    # removed for speed
    # tmp = tmp.with_columns(pl.col('paragraph').map_elements(remove_punctuation).alias('paragraph_no_pinctuation'))
    # tmp = tmp.with_columns(pl.col('paragraph_no_pinctuation').map_elements(count_spelling_errors).alias("paragraph_error_num"))
    # Calculate the length of each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    # Calculate the number of sentences and words in each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp
# feature_eng
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
# modified for speed
paragraph_fea2 = paragraph_fea
def Paragraph_Eng(train_tmp):
    num_list = [0, 50,75,100,125,150,175,200,250,300,350,400,500,600]
    num_list2 = [0, 50,75,100,125,150,175,200,250,300,350,400,500,600,700]
    aggs = [
        # Count the number of paragraph lengths greater than and less than the i-value
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_{i}_cnt") for i in [0, 50,75,100,125,150,175,200,250,300,350,400,500,600,700] ], 
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_{i}_cnt") for i in [25,49]], 
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea2],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea2],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea2],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in paragraph_fea2],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea2],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea2],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in paragraph_fea2],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in paragraph_fea2],  
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in paragraph_fea2],  
        ]
    
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df
tmp = Paragraph_Preprocess(train)
train_feats = Paragraph_Eng(tmp)
train_feats['score'] = train['score']
# Obtain feature names
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

# ⚠️ (faulty) word TfidfVectorizer ⬇️

In [ ]:
# TfidfVectorizer parameter
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(3,6),
            min_df=0.15, # modified for quicker runtime
            max_df=0.85,
            sublinear_tf=True,
)
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
train_tfid = vectorizer.fit_transform([i for i in train['full_text']])

# Convert to dataframe
df = pd.DataFrame(train_tfid.toarray())

# rename features
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

print(f'New features count: {len(tfid_columns)}')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

⬇️⬇️ Geting the feature names of the vectorizer shows that it's not word n-grams that are processed, but rather char n-grams that are counted (with spaces between chars).

This is probably due to the fact that identity (lambda x:x) is given as tokenizer, so words are not tokenized from full-text ? Tokenizer should only be overided by identity if text is already tokenized before. Perhaps vectorizer is receiving string (char sequence) instead of word sequence, so it behaves like a char ngram vectorizer...

In [ ]:
vect_feat_names=vectorizer.get_feature_names_out()
print(vect_feat_names[100:150],end='\n\n')
print(vect_feat_names[500:550],end='\n\n')
print(vect_feat_names[2000:2050],end='\n\n')

# ⚠️ (faulty) word CountVectorizer ⬇️

In [ ]:
vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,3),
            min_df=0.15,
            max_df=0.85,
)
train_tfid = vectorizer_cnt.fit_transform([i for i in train['full_text']])
dense_matrix = train_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
train_feats = train_feats.merge(df, on='essay_id', how='left')

print(f'New features count: {len(tfid_columns)}')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

In [ ]:
vect_feat_names=vectorizer_cnt.get_feature_names_out()
print(vect_feat_names[100:150],end='\n\n')
print(vect_feat_names[500:550],end='\n\n')
print(vect_feat_names[700:750],end='\n\n')

# 👌(corrected) Word n-grams TfidfVectorizer ⬇️

In [ ]:
# TfidfVectorizer parameter
vectorizer_word = TfidfVectorizer(
            #tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            #token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,6),
            min_df=0.01, # modified for words
            max_df=0.95,
            sublinear_tf=True,
)
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
train_tfid = vectorizer_word.fit_transform([i for i in train['full_text']])

# Convert to dataframe
df = pd.DataFrame(train_tfid.toarray())

# rename features
tfid_columns = [ f'tfid_word_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

print(f'New features count: {len(tfid_columns)}')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

In [ ]:
vect_feat_names=vectorizer_word.get_feature_names_out()
print(vect_feat_names[100:150],end='\n\n')
print(vect_feat_names[500:550],end='\n\n')
print(vect_feat_names[1700:1750],end='\n\n')

In [ ]:
# 
#train_feats=train_feats.drop(columns=[col for col in train_feats.columns if col.startswith('tfid_word')])
#train_feats.head(4)

# 👌(correct) Char n-grams TfidfVectorizer ⬇️

Using the **analyzer = 'char'** parameter 

In [ ]:
# TfidfVectorizer parameter for CHAR
vectorizer_char = TfidfVectorizer(
            #tokenizer=lambda x: x, # corrected
            preprocessor=lambda x: x.lower(),
            #token_pattern=None,
            strip_accents='unicode',
            analyzer = 'char',
            ngram_range=(3,6),
            min_df=0.15, 
            max_df=0.85,
            sublinear_tf=True,
)
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
train_tfid = vectorizer_char.fit_transform([i for i in train['full_text']])

# Convert to dataframe
df = pd.DataFrame(train_tfid.toarray())

# rename features
tfid_columns = [ f'tfid_char_{i}' for i in range(len(df.columns))] 

df.columns = tfid_columns

df['essay_id'] = train_feats['essay_id']
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')


print(f'new features: {len(tfid_columns)}')
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

In [ ]:
vect_feat_names=vectorizer_char.get_feature_names_out()
print(vect_feat_names[100:150],end='\n\n')
print(vect_feat_names[500:550],end='\n\n')
print(vect_feat_names[1700:1750],end='\n\n')